# Mini Curso: Introdução a Otimização Combinatória utilizando a linguagem de programação Julia

## Pacotes essenciais

In [ ]:
using Pkg 
Pkg.add("GLPK")
Pkg.add("JuMP")
Pkg.add("LinearAlgebra")

## Básicos da linguagem

In [13]:
# Impressão na tela

println("Olá mundo!")

Olá mundo!


In [ ]:
println("Insira o primeiro valor: ")
x = parse(Int64, readline())

println("Insira o segundo valor: ")
y = parse(Int64, readline())

if x < y
  println(x, " é estritamente menor que ", y)
elseif x > y
  println(x, " é estritamente maior que ", y)
else
  println(" são iguais ")
end

In [ ]:
#Como já se poderia esperar, podemos atribuir uma lista de valores a variáveis:
frutas = ["laranja", "melancia", "abacaxi"]
numeros = [42, 123]
vazio = []
println(frutas, "\n", numeros, "\n", vazio)

In [ ]:
println(frutas[1]) #Lembre-se! A contagem começa do 1

In [ ]:
#Agora suponha que queiramos criar uma array bidimensional, isto é, uma matriz
matriz = [1 2 3; 2 3 4; 2 3 4]

#Criamos uma matriz 3x3. Existem algumas funções que criam matrizes específicas, por exemplo:

ones(3,4) #Cria uma matriz 3x4 de float64 de "uns"
zeros(Int64, 5,5) #Cria uma matriz de valores inteiros de zeros 5x5
rand(1:10, 2,3) #Cria uma matriz de valores aleatórios no intervalo de 1 a 10, 2x3


In [ ]:
#=Repetição Simples (For)
Esse tipo de repetição é usado para iterar em uma lista de elementos, ou itens de um array, etc.
Iremos aqui ver algumas possibilidades de uso.
=#

for n in 1:4
    println(n)
end

In [ ]:
#Podemos também variar também em cima de um array, como por exemplo:
for i in [1, 5, 22]
    println(i)
end

In [ ]:
#O próximo exemplo mostra como percorrer um array pelos índices

lista = [1,5,22]
for i in 1:length(lista)
    println(lista[i])
end

In [70]:
notas = [5, 6, 7, 8, 7]
println("A soma das notas é: ", soma)

# soma = sum(notas)
soma = sum(notas[i] for i in 1:length(notas))

println("A média final é: ", soma/length(notas))

A soma das notas é: 33
A média final é: 6.6


## Resolvendo problemas de otimização: N-Rainhas

<p align="center">
  <img src="images//n-rainhas.png" />
</p>



![](nquee.png)

In [71]:
using GLPK
using JuMP
using LinearAlgebra

# N-Queens
N = 4

model = Model(GLPK.Optimizer);

Variável $X_{ij} \in \{0,1\}:$ 1 se uma rainha estiver na posição $(i,j)$, 0 caso contrário. 

In [72]:
@variable(model, x[i=1:N, j=1:N], Bin)

4×4 Array{VariableRef,2}:
 x[1,1]  x[1,2]  x[1,3]  x[1,4]
 x[2,1]  x[2,2]  x[2,3]  x[2,4]
 x[3,1]  x[3,2]  x[3,3]  x[3,4]
 x[4,1]  x[4,2]  x[4,3]  x[4,4]

$$\displaystyle \textrm{ max } \sum_{i = 1} ^{N} \sum_{j=1}^N X_{ij}$$

In [73]:
@objective(model, Max, sum(x[i,j] for i in 1:N, j in 1:N))

x[1,1] + x[1,2] + x[1,3] + x[1,4] + x[2,1] + x[2,2] + x[2,3] + x[2,4] + x[3,1] + x[3,2] + x[3,3] + x[3,4] + x[4,1] + x[4,2] + x[4,3] + x[4,4]

Só podemos ter exatamente uma rainha na horizontal
$$\displaystyle  \sum_{i = 1}^{N} X_{ij} \leq 1 \qquad j = 1, \dots, N$$ 

In [74]:
@constraint(model, horizontal[j in 1:N], sum(x[i,j] for i in 1:N) <= 1)

4-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1}:
 horizontal[1] : x[1,1] + x[2,1] + x[3,1] + x[4,1] <= 1.0
 horizontal[2] : x[1,2] + x[2,2] + x[3,2] + x[4,2] <= 1.0
 horizontal[3] : x[1,3] + x[2,3] + x[3,3] + x[4,3] <= 1.0
 horizontal[4] : x[1,4] + x[2,4] + x[3,4] + x[4,4] <= 1.0

Só podemos ter exatamente uma rainha na vertical
$$\displaystyle  \sum_{j = 1}^{N} X_{ij} \leq 1 \qquad i = 1, \dots, N$$ 

In [75]:
@constraint(model, vertical[i in 1:N], sum(x[i,j] for j in 1:N) <= 1)

4-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1}:
 vertical[1] : x[1,1] + x[1,2] + x[1,3] + x[1,4] <= 1.0
 vertical[2] : x[2,1] + x[2,2] + x[2,3] + x[2,4] <= 1.0
 vertical[3] : x[3,1] + x[3,2] + x[3,3] + x[3,4] <= 1.0
 vertical[4] : x[4,1] + x[4,2] + x[4,3] + x[4,4] <= 1.0

Só podemos ter exatamente uma rainha na diagonal positiva 
$$\displaystyle  \underbrace{\sum_{i = 1}^{N} \sum_{j = 1}^{N}}_{i - j = k}  X_{ij} \leq 1 \qquad k = -(N-2), \dots, N-2 \quad $$ 

<p align="center">
  <img src="images//diag-pos.jpg" />
</p>

In [76]:
@constraint(model, diag_pos[k in -(N-2):(N-2)], sum(x[i,j] for i in 1:N, j in 1:N if (i-j) == k) <= 1)

1-dimensional DenseAxisArray{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1,...} with index sets:
    Dimension 1, -2:2
And data, a 5-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1}:
 diag_pos[-2] : x[1,3] + x[2,4] <= 1.0
 diag_pos[-1] : x[1,2] + x[2,3] + x[3,4] <= 1.0
 diag_pos[0] : x[1,1] + x[2,2] + x[3,3] + x[4,4] <= 1.0
 diag_pos[1] : x[2,1] + x[3,2] + x[4,3] <= 1.0
 diag_pos[2] : x[3,1] + x[4,2] <= 1.0

Só podemos ter exatamente uma rainha na diagonal negativa

$$\displaystyle  \underbrace{\sum_{i = 1}^{N} \sum_{j = 1}^{N}}_{i + j = k}  X_{ij} \leq 1 \qquad k = 3, \dots, 2N-1 \quad $$ 

<p align="center">
  <img src="images//diag-neg.jpg" />
</p>

In [77]:
@constraint(model, diag_neg[k in 3:2*(N-1)], sum(x[i,j] for i in 1:N, j in 1:N if (i + j) == k) <= 1)

1-dimensional DenseAxisArray{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1,...} with index sets:
    Dimension 1, 3:6
And data, a 4-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1}:
 diag_neg[3] : x[2,1] + x[1,2] <= 1.0
 diag_neg[4] : x[3,1] + x[2,2] + x[1,3] <= 1.0
 diag_neg[5] : x[4,1] + x[3,2] + x[2,3] + x[1,4] <= 1.0
 diag_neg[6] : x[4,2] + x[3,3] + x[2,4] <= 1.0

In [78]:
optimize!(model)

In [79]:
println("Número de rainhas: ", objective_value(model))
solution = convert.(Int,value.(x))


4×4 Array{Int64,2}:
 0  1  0  0
 0  0  0  1
 1  0  0  0
 0  0  1  0

![](images\\sol-8-queens.png)

## Resolvendo problemas de otimização: Problema de Atribuição

- $n$ pessoas para executar $n$ tarefas distintas
- Cada pessoa deve executar uma única tarefa
- Algumas pessoas são mais aptas a executar certas tarefas.
- $c_{ij}:$ custo estimado para o indivíduo $i$ executar a tarefa $j$
  
**Objetivo:** atribuir tarefas a pessoas, ao menor custo possível.

In [84]:
c = [0 20 15 12 13
23 0 23 13 17
18 12 0 11 12
23 32 12 0 22
21 25 26 28 0]

n = 5

5

In [85]:
model = Model(GLPK.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

$\displaystyle x_{ij} \in \{0,1\}:$ atribuição da tarefa $j$ a pessoa $i$.

In [87]:
@variable(model, x[i in 1:n, j in 1:n], Bin)

5×5 Array{VariableRef,2}:
 x[1,1]  x[1,2]  x[1,3]  x[1,4]  x[1,5]
 x[2,1]  x[2,2]  x[2,3]  x[2,4]  x[2,5]
 x[3,1]  x[3,2]  x[3,3]  x[3,4]  x[3,5]
 x[4,1]  x[4,2]  x[4,3]  x[4,4]  x[4,5]
 x[5,1]  x[5,2]  x[5,3]  x[5,4]  x[5,5]

O custo das atribuições deve ser minimizado:

$$\textrm{min } \sum_{i=1}^n \sum_{j=1}^n c_{ij}x_{ij}$$

In [89]:
@objective(model, Min, sum(c[i,j]*x[i,j] for i in 1:n, j in 1:n if i != j))

20 x[1,2] + 15 x[1,3] + 12 x[1,4] + 13 x[1,5] + 23 x[2,1] + 23 x[2,3] + 13 x[2,4] + 17 x[2,5] + 18 x[3,1] + 12 x[3,2] + 11 x[3,4] + 12 x[3,5] + 23 x[4,1] + 32 x[4,2] + 12 x[4,3] + 22 x[4,5] + 21 x[5,1] + 25 x[5,2] + 26 x[5,3] + 28 x[5,4]

Cada pessoa deve efetuar uma única tarefa:
$$\sum_{j=1}^n x_{ij} = 1 \quad \forall i= 1, \dots, n$$

In [90]:
#Restrições
@constraint(model, pessoa[i in 1:n], sum(x[i,j] for j in 1:n if i != j) == 1)

5-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.EqualTo{Float64}},ScalarShape},1}:
 pessoa[1] : x[1,2] + x[1,3] + x[1,4] + x[1,5] == 1.0
 pessoa[2] : x[2,1] + x[2,3] + x[2,4] + x[2,5] == 1.0
 pessoa[3] : x[3,1] + x[3,2] + x[3,4] + x[3,5] == 1.0
 pessoa[4] : x[4,1] + x[4,2] + x[4,3] + x[4,5] == 1.0
 pessoa[5] : x[5,1] + x[5,2] + x[5,3] + x[5,4] == 1.0

Cada tarefa será efetuada por uma única pessoa:
$$\sum_{i=1}^n x_{ij} = 1 \quad \forall j = 1, \dots, n$$

In [91]:
@constraint(model, job[j in 1:n], sum(x[i,j] for i in 1:n if i!=j) == 1)

5-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.EqualTo{Float64}},ScalarShape},1}:
 job[1] : x[2,1] + x[3,1] + x[4,1] + x[5,1] == 1.0
 job[2] : x[1,2] + x[3,2] + x[4,2] + x[5,2] == 1.0
 job[3] : x[1,3] + x[2,3] + x[4,3] + x[5,3] == 1.0
 job[4] : x[1,4] + x[2,4] + x[3,4] + x[5,4] == 1.0
 job[5] : x[1,5] + x[2,5] + x[3,5] + x[4,5] == 1.0

In [92]:
optimize!(model)

In [96]:
function print_solution()
    for i in 1:n
        for j in 1:n
            if value(x[i,j]) == 1
                println("Pessoa ", i, " foi atribuída ao trabalho: ", j, " com custo: ", c[i,j])
            end
        end
    end
    println("O custo ótimo foi: ", objective_value(model))
end

print_solution()

Pessoa 1 foi atribuída ao trabalho: 5 com custo: 13
Pessoa 2 foi atribuída ao trabalho: 4 com custo: 13
Pessoa 3 foi atribuída ao trabalho: 2 com custo: 12
Pessoa 4 foi atribuída ao trabalho: 3 com custo: 12
Pessoa 5 foi atribuída ao trabalho: 1 com custo: 21
O custo ótimo foi: 71.0


## Resolvendo problemas de otimização: Problema da Mochila 0-1

### Agora é com você. Desafio!

  - Temos uma mochila com uma capacidade máxima (peso máximo) dado por $W$
  - Imagine que temos $n$ possíveis itens que podem entrar na mochila
  - Cada item $i$ possui um peso $a_i$ específico e seu preço é $c_i$.

**Objetivo: Selecionar um subconjunto de itens que dê o máximo retorno possível, isto é, que o custo total do que está dentro da mochila seja o maior possível!**

**Variáveis**

$x_i \in \{0,1\}:$ pegar ou não o item $i$ para colocar na mochila

**Função Objetivo**

$$\textrm{max } \sum_{i=1}^n c_ix_i$$

**Restrições**

- Não é possível colocar mais itens do que a capacidade $W$ dada.

$$\sum_{i=1}^n a_ix_i \leq W$$

$$x_i \in \{0,1\}$$

In [24]:
n = 5 # Número de itens 
W = 50 # Peso máximo que a mochila pode carregar

a = [20 12 17 13 17] # Capacidade de cada item
c = [30 11 20 18 17] # Custo de cada item 

1×5 Array{Int64,2}:
 30  11  20  18  17